# Filtragem Colaborativa por Usuário

## Introdução

A filtragem colaborativa faz recomendações com base em padrões de comportamento passado de vários usuários, sem necessitar de qualquer informação adicional sobre os itens ou usuários. A ideia básica da filtragem colaborativa é criar uma matriz usuário-item. O conjunto de dados pode ser representado como uma matriz onde as linhas correspondem aos usuários e as colunas aos filmes. 

Matrix | Inception | Titanic | Star Wars | The Godfather
--------|-----------|---------|-----------|--------------
Alice    |     5     |    3    |     4     |      0
Bob      |     4     |    0    |     5     |      3
Carol    |     3     |    5    |     4     |      4
Dave     |     0     |    2    |     0     |      5
Eve      |     2     |    5    |     0     |      4

## Ingestão dos dados

**Bibliotecas**

In [2]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

In [15]:
# Dados de entrada
data = {
    'Inception': [5, 4, 3, np.nan, 2],
    'Titanic': [3, np.nan, 5, 2, 5],
    'Star Wars': [4, 5, 4, np.nan, np.nan],
    'The Godfather': [np.nan, 3, 4, 5, 4]
}

# Cria um dataframe do pandas
df = pd.DataFrame(data, index=['Alice', 'Bob', 'Carol', 'Dave', 'Eve'])
df.head(2)

,Inception,Titanic,Star Wars,The Godfather
Alice,5.0,3.0,4.0,NaN
Bob,4.0,NaN,5.0,3.0


## Preparação dos Dados

In [14]:
# Substituir NaN por 0
df_filled = df.fillna(0)
df_filled

,Inception,Titanic,Star Wars,The Godfather
Alice,5.0,3.0,4.0,0.0
Bob,4.0,0.0,5.0,3.0
Carol,3.0,5.0,4.0,4.0
Dave,0.0,2.0,0.0,5.0
Eve,2.0,5.0,0.0,4.0


Normalização das avaliações. Essa normalização ajuda a lidar com o viés de que diferentes usuários podem ter diferentes escalas de classificação. Por exemplo, um usuário pode ser geralmente mais crítico e dar classificações mais baixas, enquanto outro pode dar classificações mais altas. Normalizar os dados dessa maneira permite que o sistema de recomendação lide melhor com essas diferenças de escala.

In [13]:
# Normalizar os dados subtraindo a média de cada usuário
normalized_df = df_filled.sub(df_filled.mean(axis=1), axis=0)
normalized_df

,Inception,Titanic,Star Wars,The Godfather
Alice,2.00,0.00,1.00,-3.00
Bob,1.00,-3.00,2.00,0.00
Carol,-1.00,1.00,0.00,0.00
Dave,-1.75,0.25,-1.75,3.25
Eve,-0.75,2.25,-2.75,1.25


## Desenvolvimento do Modelo

### KNN

In [11]:
# Usar o algoritmo KNN com a métrica de similaridade do cosseno
knn = NearestNeighbors(metric='cosine', n_neighbors=3, n_jobs=-1)

# Ajustar o modelo com os dados normalizados
knn.fit(normalized_df)

# Calcular as distâncias e os índices dos vizinhos mais próximos para todos os usuários
distances, indices = knn.kneighbors(normalized_df)

## Função de Recomendação

In [12]:
# Prevê a classificação que um determinado usuário (especificado pelo `user_index`) daria a um filme específico (`movie_name`).
def predict_rating(user_index, movie_name, data, indices):
    # Seleciona os índices dos usuários mais próximos (vizinhos) para o usuário alvo. O slicing `[1:]` é usado para ignorar o próprio usuário, que é sempre retornado como o 'vizinho' mais próximo quando se usa o algoritmo KNN do `scikit-learn`
    neighbor_indices = indices[user_index, 1:]  # Ignora o próprio usuário

    # Obtém as classificações que esses vizinhos deram para o filme em questão. 
    # Isso é feito selecionando a coluna correspondente ao `movie_name` do DataFrame `data` e, em seguida, usando os índices dos vizinhos para obter suas classificações.
    neighbor_ratings = data.loc[:, movie_name].iloc[neighbor_indices]

    # Calcular a média das classificações dos vizinhos
    predicted_rating = neighbor_ratings.mean()

    return predicted_rating

# Prever a classificação para Alice para 'The Godfather'
predicted_rating_for_alice = predict_rating(0, 'The Godfather', df, indices)

predicted_rating_for_alice

3.5